# Lesson 17 - Solutions

In [1]:
import pandas as pd
from pandas.io import sql

Pandas can be used to connect to most relational databases. In this demonstration, we will create and connect to a SQLite database. SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

If you are looking to start using a database without the setup of `mysql` or `postgres`, SQLite is a good start.

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect("dat-test.db")

Let's return to the Rossmann sales data and load that into the database.

Data is moved to the database through the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arugments:
- `name`, the table name to create
- `con`, a connection to a database
- `index`, whether to input the index column
- `schema`, if we want to write a custom schema for the new table
- `if_exists`, what to do if the table already exists. We can overwrite it, add to it or fail

In [4]:
data = pd.read_csv("../../../../data/rossmann.csv", low_memory = False)
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [5]:
data.to_sql("rossmann_sales", con = conn, if_exists = "replace", index = False)

In [6]:
query = """
SELECT * 
FROM   ROSSMANN_SALES 
LIMIT  10
"""
sql.read_sql(query, con = conn)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
5,6,5,2015-07-31,5651,589,1,1,0,1
6,7,5,2015-07-31,15344,1414,1,1,0,1
7,8,5,2015-07-31,8492,833,1,1,0,1
8,9,5,2015-07-31,8565,687,1,1,0,1
9,10,5,2015-07-31,7185,681,1,1,0,1


### CHECK: Load the Rossmann Store metadata in `rossmann-stores.csv` and create a table into the database from it

In [7]:
rossmann_stores = pd.read_csv("../../../../data/rossmann-stores.csv")
rossmann_stores.to_sql("rossmann_stores", if_exists = "replace", index = False, con = conn)

### CHECK: (SELECT) Have the students write a query that returns the `Store`, `Date` and `Customers`

In [8]:
query = """
SELECT STORE, DATE, CUSTOMERS
FROM   ROSSMANN_SALES
"""
sql.read_sql(query, con = conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


### CHECK: (WHERE) Have the students write a query that returns the `Store`, `Date` and `Customers` for when the stores were open and running a promotion

In [9]:
query = """
SELECT STORE, DATE, CUSTOMERS
FROM   ROSSMANN_SALES
WHERE  OPEN = 1 AND PROMO = 1
"""
sql.read_sql(query, con = conn).head()

,Store,Date,Customers
0,1,2015-07-31,555
1,2,2015-07-31,625
2,3,2015-07-31,821
3,4,2015-07-31,1498
4,5,2015-07-31,559


### CHECK: (GROUP BY) Have the students write a query that returns the total sales on promotion days

In [10]:
query = """
SELECT   PROMO, STORE, SUM(SALES)
FROM     ROSSMANN_SALES
GROUP BY PROMO
"""
sql.read_sql(query, con = conn).head()

,Promo,Store,SUM(SALES)
0,0,1115,2771974337
1,1,1115,3101206286


## Exercises
1. Load the Walmart sales and store features data
1. Create a table for each of those datasets
1. Select the store, date and fuel price on days it was over 90 degrees
1. Select the store, date and weekly sales and temperature
1. What were average sales on holiday vs. non-holiday sales
1. What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees

In [11]:
# Load the Walmart sales and store features data
walmart_sales = pd.read_csv("../../../../data/walmart-sales.csv")
walmart_sales.head()

walmart_features = pd.read_csv("../../../../data/features.csv")
walmart_features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [12]:
# Create a table for each of those datasets
walmart_sales.to_sql("walmart_sales", index = False, if_exists = "replace", con = conn)
walmart_features.to_sql("walmart_features", if_exists = "replace", index = False, con = conn)

In [13]:
# Select the store, date and fuel price on days it was over 90 degrees
query = """
SELECT DATE, STORE, FUEL_PRICE, TEMPERATURE
FROM   WALMART_FEATURES
WHERE  TEMPERATURE > 90
"""
sql.read_sql(query, con = conn).head()

,Date,Store,Fuel_Price,Temperature
0,2011-08-05,1,3.684,91.65
1,2011-08-12,1,3.638,90.76
2,2011-07-29,2,3.682,90.07
3,2011-08-05,2,3.684,93.34
4,2011-08-12,2,3.638,91.58


In [14]:
# Select the store, date and weekly sales and temperature
query = """
SELECT S.STORE, S.DATE, S.WEEKLY_SALES, F.TEMPERATURE
FROM   WALMART_SALES AS S
JOIN   WALMART_FEATURES AS F
ON     S.STORE = F.STORE AND S.DATE = F.DATE
"""
sql.read_sql(query, con = conn).head()

,Store,Date,Weekly_Sales,Temperature
0,1,2010-02-05,24924.50,42.31
1,1,2010-02-12,46039.49,38.51
2,1,2010-02-19,41595.55,39.93
3,1,2010-02-26,19403.54,46.63
4,1,2010-03-05,21827.90,46.50


In [15]:
# What were average sales on holiday vs. non-holiday sales
query = """
SELECT   ISHOLIDAY, AVG(WEEKLY_SALES)
FROM     WALMART_SALES AS S
GROUP BY ISHOLIDAY
"""
sql.read_sql(query, con = conn).head()

,IsHoliday,AVG(WEEKLY_SALES)
0,0,15901.445069
1,1,17035.823187


In [16]:
# What were average sales on holiday vs. non-holiday sales when the temperature was below 32 degrees
query = """
SELECT   S.ISHOLIDAY, AVG(S.WEEKLY_SALES)
FROM     WALMART_SALES AS S
JOIN     WALMART_FEATURES AS F
ON       S.STORE = F.STORE AND S.DATE = F.DATE
WHERE    F.TEMPERATURE < 32
GROUP BY S.ISHOLIDAY
"""
sql.read_sql(query, con = conn).head()

,IsHoliday,AVG(S.WEEKLY_SALES)
0,0,15275.770307
1,1,15111.471238
